In [1]:
from discussion_agents.cog.functional.self_refine import _build_agent_prompt
from discussion_agents.cog.prompts.self_refine import GSM8K_FEWSHOT_EXAMPLES

In [2]:
question_prefix="# Q: "
answer_prefix="# solution using Python:\n"
intra_example_sep="\n"
question = "A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?"

prompt = _build_agent_prompt(
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES,
    question_prefix=question_prefix,
    intra_example_sep=intra_example_sep,
    answer_prefix=answer_prefix
)

In [3]:
print(prompt)

# Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
# solution using Python:

def solution():
    """Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?"""
    jason_lollipops_initial = 20
    jason_lollipops_after = 12
    denny_lollipops = jason_lollipops_initial - jason_lollipops_after
    result = denny_lollipops
    return result



# Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
# solution using Python:

def solution():
    """There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?"""
    trees_initial = 15
    trees_after = 21
    trees_added = trees_after - trees_initial
  